# RF Data Converter Introduction
----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents an introduction to the RF Data
Converters (RF DCs) on the ZCU208 board.

## Aims
* Describe the `xrfdc` Python package that is required to control and
  configure the RF DCs from Jupyter
* Investigate the radio hierarchy in the base overlay, allowing the
  user to develop very simple RF designs. 
* Present the data inspection and visualisation of the RF DCs using 
  Plotly.

## Table of Contents
* [Introduction](#introduction)
* [Hardware Setup](#hardware-setup)
* [The Radio Hierarchy](#radio-hierarchy)
* [Transmitter and Receiver](#the-transmitter-and-receiver)
* [RF Data Inspection](#rf-data-inspection)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)
* [Xilinx, Inc, "Vivado Design Suite: The AXI Reference Guide", UG1037, v4.0, June 2017](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf)
* [Xilinx, Inc, "ZCU208 Evaluation Board User Guide", UG1410, v1.0, July 2020](https://www.xilinx.com/support/documentation/boards_and_kits/zcu208/ug1410-zcu208-eval-bd.pdf)
* [Xilinx, Inc, "CLK104 RF Clock Add-on Card", UG1437, v1.0, March 2020](https://japan.xilinx.com/support/documentation/boards_and_kits/zcu216/ug1437-clk104.pdf)

## Revision History

* v1.0 | 02/02/2021 | First notebook revision.
* v2.0 | 23/09/2021 | Revised for the ZCU208.
* v3.0 | 27/09/2022 | Additional ADC tiles added to overlay.

----

## Introduction <a class="anchor" id="introduction"></a>

An overview of the Zynq RFSoC device is shown below; there are
three major components:

* Processing System (PS)
* Programmable Logic (PL)
* RF Data Converters (RF DC, including ADC and DAC)

The RF data converters are significant features of the Zynq RFSoC
device as they interface directly to the PL.
This brings many advantages, including the ability to
perform direct RF sampling and low-latency processing.

The ZCU208 is a Gen 3 RFSoC platform consisting of an XCZU48DR-FSVG1517-2-E RFSoC device, 
which contains the following RF DCs:

* 8x RF Analogue-to-Digital Converters (RF ADCs)
    * Sample rate of 5 GSPS
    * 14-bit conversion
* 8x RF Digital-to-Analogue Converters (RF DACs)
    * Sample rate of 7 GSPS
    * 14-bit conversion

Let's take a closer look at the RF DC block. Starting with
the RF ADC tiles, you will notice that there are 4 tiles on this
particular RFSoC device. Each tile contains 2 RF ADC blocks, 
which can be used to receive, or Digital Down Convert (DDC), 
an analogue signal. Each RF ADC block contains:

* A gearbox FIFO
* 1x (bypass), 2x, 3x, 4x, 5x, 6x, 8x, 10x, 12x, 16x, 20x, 24x or 40x decimator
* A complex mixer
* A Quadrature Modulation Correction (QMC) unit
* Embedded Digital Step Attenuator (DSA)
* On-chip clock distribution network

The image below presents the typical RF ADC processing 
pipeline:

1. The RF ADC samples the input waveform to convert it
   into a digital signal.
2. A threshold detector can be employed to detect and record
   input amplitude levels. 
3. The QMC is used to offset potential imbalance in the
   quality of the received complex signal.
4. The I and Q mixer can mix the input signal to baseband.
5. The I and Q decimators are capable of a wide range of
   decimation options before interfacing the PL via the gearbox FIFO.

<img src='data/rfsystem_rfadc_block_overview.png' align='left' style='left' height='75%' width='75%'/>

The RF DAC block has a similar pipeline to the RF ADC;
however, it is in reverse this time for Digital Up Conversion (DUC).

1. The data to be transmitted is interfaced to the RF DAC block from the PL.
2. The digital signal is interpolated by a factor of 1x (bypass), 2x, 3x, 4x, 5x, 6x, 8x, 10x, 12x, 16x, 20x, 24x    or 40x.
3. The digital signal is transferred to the complex mixer to be mixed
   to the desired carrier frequency. 
4. After the QMC and coarse delay block, the signal can optionally
   be convolved with an inverse (anti) sinc filter to improve the
   roll-off of the first Nyquist zone.
5. The digital signal is then converted to analogue through the
   RF DAC sampler.


<img src='data/rfsystem_rfdac_block_overview.png' align='left' style='left' height='75%' width='75%'/>

In this notebook, we will be demonstrating many of the features
mentioned above via a simple loopback example.

# Hardware Setup <a class="anchor" id="hardware-setup"></a>

The CLK104 add on board should be connected to the RFDCs using SMCs. This will enable a connection from the LMX to the on tile PLLs associated with ADC tile 225 and DAC tile 230. Clock distribution will be used to provide this clock to the other active tiles.
<img src='data/CLK104_LMX_to_RFSOC.JPG' align='middle' width='50%' height='50%'/>

Your ZCU208 board is capable of 8 channels, dependent on hardware configuration. Loopback is possible through the use of the XM655 add on card included with the ZCU208 kit. There are 8 JHC connections from the RFDC to the break out board. For this demonstration we will be using 2 DAC channels (JHC3) as the source.  Four ADC channels are configured but only 2 ADC channels (JHC5, JHC7) are captured at a time. You should create the loopback connection using SMA cables as shown below.  The DC Block and male-to-male adapters are necessary to complete the connections and are shown further below.

## Configuration One
* DAC Tile 230 (channel 2) to ADC Tile 224 (channel 0)
* DAC Tile 231 (channel 3) to ADC Tile 225 (channel 1)

## Configuration Two
* DAC Tile 230 (channel 2) to ADC Tile 226 (channel 2)
* DAC Tile 231 (channel 3) to ADC Tile 227 (channel 3)

<img src='data/DAC_to_ADC_both.JPG' align='middle' width='70%' height='70%'/>
<BR>
<img src='data/DAC_to_ADC.JPG' align='middle' width='70%' height='70%'/>

Switching from configuration one to two is a matter of removing the Carlisle connector from the ADC pad JHC5 to JHC7 and vice versa while leaving the DAC pad on JHC3.
    

<img src='data/DAC_to_ADC_M2M_DCBlock.JPG' align='middle' width='70%' height='70%'/>
DC Block and male to male adapter

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

The Carslile SMA cabling is prone to cable kinking, twisting and bending.  Take care to use the SMA housing only to screw and unscrew which helps avoid twisting the cable.  The various male connectors J21, et al. must not contact the SMA metal leads otherwise a short may occur.
    
In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## The Radio Hierarchy <a class="anchor" id="radio-hierarchy"></a>

We need to add custom control logic in the PL to communicate
with the RF ADC or RF DAC. The ZCU208 base overlay contains
such logic that will allow you to evaluate the RF DCs. 
There are two IP Cores - the transmitter and the receiver.

* **Amplitude Controller** - can apply a value to the input of the RF DAC.
The value can be set by writing to an AXI-Lite register. 
The register value can be mixed with the Numerically
Controlled Oscillator (NCO) in the fine mixer to create a tone. 
The tone is looped back round into the RF ADC.

* **Packet Generator** - interfaces to the output of RF ADC block. 
The data is used to generate an [AXI-Stream packet](https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf).
The AXI-Stream packet is sent to an AXI Direct Memory
Access (DMA) core and transferred to DDR memory.

These 2 IP cores are connected to each channel of the ZCU208 board.
They are conveniently placed inside a hierarchical block, `radio`, 
as shown below.

Let's now download the base overlay and initialize the drivers.

In [1]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as ipw
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

The ZCU208 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs). The base overlay
has a simple method to initialize these clocks. Run the cell below to set
the LMK and LMX clocks to 245.76MHz and 491.52MHz, respectively.

In [2]:
base.init_rf_clks()

In [3]:
base.radio

Now we should investigate the `radio` hierarchy.

In [6]:
base.radio?

Type:           RadioWrapper
String form:    <rfsystem.hierarchies.RadioWrapper object at 0xffff78c38730>
File:           /usr/local/share/pynq-venv/lib/python3.10/site-packages/rfsystem/hierarchies.py
Docstring:     
Hierarchy driver for the entire radio subsystem.

Exposes the transmit, receive, and rfdc as attributes. The rfdc
dac and adc tiles are passed to the transmitter and receiver
respectively.

This wrapper assumes the following separate pipelines
* transmitter -> rfdc -> "the outside world"
* "the outside world" -> rfdc => receiver

The rfdc core is common between each pipeline.

Attributes
----------
transmitter : pynq.lib.radio.Transmitter
    The transmit control system
receiver : pynq.lib.radio.Receiver
    The receive control system
rfdc : xrfdc package
    The RF Data Converter core
Init docstring: Create a new _IPMap based on a hierarchical description.


We can see the `transmitter` and `receiver` hierarchies. 
The RF DC object is also available, however, we don't need this. The reason
is that each transmitter and receiver is automatically paired with their
associated RF DC tile and block.

The `radio` hierarchy also initializes all of the active RF ADC and RF DAC
blocks to sample at 4915.2 GSPS (close to the maximum sample rate for the RF ADC).
By default the RF DAC and RF ADC mixer frequencies are 1228.8 MHz, which is
in the center of the first Nyquist Zone.

## Transmitter and Receiver <a class="anchor" id="the-transmitter-and-receiver"></a>

The transmitter hierarchy contains two channels. Each channel contains an
*Amplitude Controller* connected directly to an RF DAC. We can list the
available channels of the transmitter using the `get_channel_description()`
method.

In [5]:
base.radio.transmitter.get_channel_description??

Signature: base.radio.transmitter.get_channel_description()
Docstring:
Obtains the channels and their tile and block for the user.
        
Source:   
    def get_channel_description(self):
        """Obtains the channels and their tile and block for the user.
        """
        ch_dict = {}
        for i in range(0, len(self._dac_list)):
            tile = self._dac_list[i][0]
            block = self._dac_list[i][1]
            ch_dict.update({''.join(["Channel ", str(i)]): 
                            {"Tile": tile+228,
                             "Block": block}})
        return ch_dict
File:      /usr/local/share/pynq-venv/lib/python3.10/site-packages/rfsystem/hierarchies.py
Type:      method


The report above states that there are two active channels in the system.

The receiver hierarchy also contains two channels. The *Packet Generator*
arranges the data into AXI-Stream packets and sends these packets to DDR
memory via AXI DMA. Let's run the cell below.

In [7]:
base.radio.receiver.get_channel_description()

{'Channel 0': {'Tile': 224, 'Block': 0},
 'Channel 1': {'Tile': 225, 'Block': 0},
 'Channel 2': {'Tile': 226, 'Block': 0},
 'Channel 3': {'Tile': 227, 'Block': 0}}

There are several methods to read and write to RF block registers.
These methods are common to the RF DAC and RF ADC.

```python
BlockStatus
MixerSettings
QMCSettings
CoarseDelaySettings
NyquistZone
```

For example, `base.radio.transmitter.channel[0].dac_block.QMCSettings`
will display the QMC settings for the transmit DAC block.

For many of these settings, you can change them and they will impact the operation.

In [8]:
base.radio.transmitter.channel[0].dac_block.QMCSettings

{'EnablePhase': 0,
 'EnableGain': 0,
 'GainCorrectionFactor': 0.0,
 'PhaseCorrectionFactor': 0.0,
 'OffsetCorrectionFactor': 0,
 'EventSource': 0}

Another example you can try:
```python
base.radio.receiver.channel[0].adc_block.CoarseDelaySettings
```

To further investigate the RF ADC and RF DAC, you can run the code cell below.

In [9]:
help(base.radio.transmitter.channel[0].dac_block)

Help on RFdcDacBlock in module xrfdc object:

class RFdcDacBlock(RFdcBlock)
 |  RFdcDacBlock(*args, **kwargs)
 |  
 |  Method resolution order:
 |      RFdcDacBlock
 |      RFdcBlock
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  SetDACVOP(self, uACurrent)
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  FabRdVldWords
 |  
 |  OutputCurr
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  DACCompMode
 |  
 |  DataPathMode
 |  
 |  DecoderMode
 |  
 |  FabWrVldWords
 |  
 |  IMRPassMode
 |  
 |  InterpolationFactor
 |  
 |  InvSincFIR
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from RFdcBlock:
 |  
 |  GetConnectedIData(self)
 |  
 |  GetConnectedQData(sel

More to try:
```python
help(base.radio.transmitter.channel[0].dac_tile)
help(base.radio.receiver.channel[0].adc_block)
help(base.radio.receiver.channel[0].adc_tile)
```
Many of these attributes / methods depend on the configuration
of the PL and the hardware. You will need to implement your own hardware 
design if you want to do something else.

## RF Data Inspection <a class="anchor" id="rf-data-inspection"></a>

Now that you have a fundamental understanding of how to control the RF DCs, 
we can inspect RF data using the radio hierarchy block. To make it more
interesting, we generate stimulus using the RF DAC block for each channel. 
Let's configure the mixer frequencies for each RF-DAC block channel below. 

We choose frequencies which fall directly within a frequency bin with no scallop loss, where:

$$
\frac{f_s}{samples} = f_{bin}\\
\frac{2457.6 MHz}{256} = 9.6 MHz
$$

In [5]:
#base.radio.transmitter.channel[0].dac_block.MixerSettings['Freq'] = 768  # MHz Unused 
#base.radio.transmitter.channel[1].dac_block.MixerSettings['Freq'] = 960  # MHz Unused
base.radio.transmitter.channel[2].dac_block.MixerSettings['Freq'] = 1056 # MHz 
base.radio.transmitter.channel[3].dac_block.MixerSettings['Freq'] = 1200 # MHz

The PL contains an amplitude controller for signal transmission. 
We can set the amplitude of the signal and enable transmission for each channel.

In [6]:
for i in range(0, len(base.radio.transmitter.channel)):
    base.radio.transmitter.channel[i].control.gain = 0.8
    base.radio.transmitter.channel[i].control.enable = True

The RF-ADC block mixer frequency can also be tuned. We set it to 1228.8 MHz as
that is the default setting. This is a suitable mixer frequency for the
next part of the demonstration, where we will transfer time domain samples
from receiver channels.

Firstly, we will define the numbers of samples to be transferred. 
The range of samples that can be transferred is between 16 to 32768.
Running the code cell below, will transfer complex data, with an amplitude of 1.

In [7]:
number_samples = 256
cdata = []
for i in range(0, len(base.radio.receiver.channel)):
    cdata.append(base.radio.receiver.channel[i].transfer(number_samples))

Now that we have complex channel data in Jupyter, we can import plotly, numpy,
and ipwidgets to enable data visualization.

The data can be plotted using `go.Scatter` and `go.FigureWidget` for each channel. Ipywidgets allows each figure to be displayed at the same time.

In [8]:
sample_frequency = 2457.6e6
figs = []

for i in range(0, len(base.radio.receiver.channel)):
    plt_re_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.real(cdata[i]), name='Real'))
    plt_im_temp = (go.Scatter(
            x = np.arange(0, number_samples/sample_frequency, 
                          1/sample_frequency),
            y = np.imag(cdata[i]), name='Imag'))
    figs.append(go.FigureWidget(data = [plt_re_temp, plt_im_temp],
            layout = {'title': ''.join(['Time Domain Plot of ADC Channel ', 
                                         str(i)]), 
                      'xaxis': {'title' : 'Seconds (s)',
                                'autorange' : True},
                      'yaxis' : {'title' : 'Amplitude (V)'}}))

ipw.VBox(figs)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

Repeat the cells above but changing to Configuration Two.  With Configuration Two, ADC channels 2 and 3 should now have a discernable sinusoidal waveform.

You can also return to the [RF Data Inspection](#rf-data-inspection) section
and try different mixer frequency or different amplitude gain.

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a simple introduction to the RF Data Converters. 
The RF DCs were briefly described and a radio hierarchy in PL was explored.
RF data visualization was performed using `plotly`, `ipywidgets`, and `numpy` in Jupyter.

| | [RF Spectrum Analysis ➡️](02_rf_spectrum_analysis.ipynb)

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----